In [ ]:
!pip install konlpy

In [ ]:
!pip install pandas openpyxl xlrd konlpy scikit-learn

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from konlpy.tag import Okt
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K

# 파일 불러오기
file_path = '/content/drive/MyDrive/movie14.xls'
data = pd.read_excel(file_path, engine='xlrd', header=None)

# 컬럼 이름 지정
column_names = [
    "movie_name", "movie_name_English", "year_of_production",
    "country_of_production", "type", "genre", "production_status",
    "director", "production_company"
]

data.columns = column_names

# NLP를 위한 텍스트 특성 결합
data['Text'] = data['movie_name'] + ' ' + data['production_company'] + ' ' + data['genre'] + ' ' + data['director']

# 한국어 텍스트 처리를 위한 Okt 초기화
okt = Okt()

# 텍스트를 토큰화하고 원형으로 복원하는 함수 정의
def tokenize_and_lemmatize(text):
    if isinstance(text, str):
        tokens = okt.morphs(text, stem=True)
        return ' '.join(tokens)
    else:
        return ''

# 텍스트 데이터에 함수 적용
data['Processed_Text'] = data['Text'].apply(tokenize_and_lemmatize)

# 텍스트 데이터 인코딩 및 패딩
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['Processed_Text'])
sequences = tokenizer.texts_to_sequences(data['Processed_Text'])
max_sequence_length = max(len(seq) for seq in sequences)
X = pad_sequences(sequences, maxlen=max_sequence_length)

# 레이블 인코딩
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['movie_name'])

# 레이블을 원-핫 인코딩으로 변환
y = to_categorical(y)

# 모델 구축
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=256, input_length=max_sequence_length))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.3))  # Dropout 비율 낮춤
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(32, activation='relu', kernel_regularizer='l2'))
model.add(Dropout(0.3))  # Dropout 비율 낮춤
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

# RMSE 계산 함수 정의
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

# 모델 컴파일 (RMSE를 손실 함수로 사용)
model.compile(loss=rmse, optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

# Early Stopping 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 모델 학습
history = model.fit(X, y, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# 예측 함수를 tf.function으로 정의
@tf.function
def predict(model, sequence):
    return model(sequence, training=False)

def get_recommendations_dl(movie_title=None, genre=None, country=None, year=None):
    if not movie_title and not genre and not country and not year:
        return "영화 제목, 장르, 제작 국가 또는 제작 연도 중 하나를 입력하세요."

    if movie_title:
        if movie_title not in data['movie_name'].values:
            return "영화 제목이 데이터베이스에 없습니다."

        # 영화 제목에 해당하는 인덱스 가져오기
        idx = data[data['movie_name'] == movie_title].index[0]
        movie_sequence = pad_sequences(tokenizer.texts_to_sequences([data.loc[idx, 'Processed_Text']]), maxlen=max_sequence_length)
        movie_sequence = tf.convert_to_tensor(movie_sequence)  # 텐서로 변환
        predictions = predict(model, movie_sequence)[0].numpy()

        sim_scores = []
        for i in range(len(predictions)):
            sim_scores.append((i, predictions[i]))

        # 유사도 순으로 정렬
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # 상위 5개 추천
        sim_scores = sim_scores[:5]
        movie_indices = [i[0] for i in sim_scores]
        recommendations = data.iloc[movie_indices]

        # 시각화를 위한 유사도 점수 포함
        similarity_scores = [i[1] for i in sim_scores]
        return recommendations[['movie_name', 'genre', 'country_of_production', 'year_of_production']].values.tolist(), similarity_scores

    else:
        # 필터링된 영화에서 상위 5개 선택
        recommendations = data.head(5)
        similarity_scores = [1] * len(recommendations)  # 필터링된 결과에서는 유사도 점수를 1로 설정

    recommendations = recommendations.copy()
    recommendations.loc[:, 'year_of_production'] = recommendations['year_of_production'].astype(int)
    return recommendations[['movie_name', 'genre', 'country_of_production', 'year_of_production']].values.tolist(), similarity_scores

# 사용자 입력을 통한 영화 제목, 장르, 제작 국가, 제작 연도 입력
user_input = input("영화 제목, 장르, 제작 국가, 제작 연도를 쉼표로 구분하여 입력하세요 (예: 제목,장르,국가,연도): ").strip()

# 입력된 값을 쉼표로 구분하여 리스트로 변환
inputs = [i.strip() if i.strip() else None for i in user_input.split(',')]
inputs = inputs + [None] * (4 - len(inputs))  # 입력된 값의 개수가 4개가 되도록 보충
movie_title, genre, country, year = inputs

recommended_movies, similarity_scores = get_recommendations_dl(movie_title, genre, country, year)
print("추천된 영화들:\n", recommended_movies)


Epoch 1/100
95/95 [==============================] - 44s 327ms/step - loss: 0.4253 - accuracy: 3.3102e-04 - val_loss: 0.4019 - val_accuracy: 0.0013
Epoch 2/100
95/95 [==============================] - 24s 251ms/step - loss: 0.3808 - accuracy: 0.0000e+00 - val_loss: 0.3598 - val_accuracy: 0.0013
Epoch 3/100
95/95 [==============================] - 25s 264ms/step - loss: 0.3408 - accuracy: 0.0000e+00 - val_loss: 0.3220 - val_accuracy: 0.0013
Epoch 4/100
95/95 [==============================] - 25s 267ms/step - loss: 0.3050 - accuracy: 3.3102e-04 - val_loss: 0.2880 - val_accuracy: 0.0013
Epoch 5/100
95/95 [==============================] - 26s 279ms/step - loss: 0.2727 - accuracy: 3.3102e-04 - val_loss: 0.2574 - val_accuracy: 0.0013
Epoch 6/100
95/95 [==============================] - 25s 264ms/step - loss: 0.2437 - accuracy: 3.3102e-04 - val_loss: 0.2299 - val_accuracy: 0.0013
Epoch 7/100
95/95 [==============================] - 29s 302ms/step - loss: 0.2176 - accuracy: 3.3102e-04 - val_

ValueError: cannot convert float NaN to integer

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from konlpy.tag import Okt
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

# 파일 불러오기
file_path = '/content/drive/MyDrive/movie14.xls'
data = pd.read_excel(file_path, engine='xlrd', header=None)

# 컬럼 이름 지정
column_names = [
    "movie_name", "movie_name_English", "year_of_production",
    "country_of_production", "type", "genre", "production_status",
    "director", "production_company"
]

data.columns = column_names

# NLP를 위한 텍스트 특성 결합
data['Text'] = data['movie_name'] + ' ' + data['production_company'] + ' ' + data['genre'] + ' ' + data['director']

# 한국어 텍스트 처리를 위한 Okt 초기화
okt = Okt()

# 텍스트를 토큰화하고 원형으로 복원하는 함수 정의
def tokenize_and_lemmatize(text):
    if isinstance(text, str):
        tokens = okt.morphs(text, stem=True)
        # 불용어 제거 (예시: 단어 길이가 1 이하인 경우 제거)
        tokens = [token for token in tokens if len(token) > 1]
        return ' '.join(tokens)
    else:
        return ''

# 텍스트 데이터에 함수 적용
data['Processed_Text'] = data['Text'].apply(tokenize_and_lemmatize)

# 텍스트 데이터 인코딩 및 패딩
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['Processed_Text'])
sequences = tokenizer.texts_to_sequences(data['Processed_Text'])
max_sequence_length = max(len(seq) for seq in sequences)
X = pad_sequences(sequences, maxlen=max_sequence_length)

# 레이블 인코딩
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['movie_name'])

# 레이블을 원-핫 인코딩으로 변환
y = to_categorical(y)

# 모델 구축
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=256, input_length=max_sequence_length))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.3))  # Dropout 비율 유지
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(32, activation='relu', kernel_regularizer='l2'))
model.add(Dropout(0.3))  # Dropout 비율 유지
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

# RMSE 계산 함수 정의
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

# 모델 컴파일 (RMSE를 손실 함수로 사용)
model.compile(loss=rmse, optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

# Early Stopping 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# 모델 학습
history = model.fit(X, y, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# 예측 함수를 tf.function으로 정의
@tf.function
def predict(model, sequence):
    return model(sequence, training=False)

def get_recommendations_dl(movie_title=None, genre=None, country=None, year=None):
    if not movie_title and not genre and not country and not year:
        return "영화 제목, 장르, 제작 국가 또는 제작 연도 중 하나를 입력하세요."

    if movie_title:
        if movie_title not in data['movie_name'].values:
            return "영화 제목이 데이터베이스에 없습니다."

        # 영화 제목에 해당하는 인덱스 가져오기
        idx = data[data['movie_name'] == movie_title].index[0]
        movie_sequence = pad_sequences(tokenizer.texts_to_sequences([data.loc[idx, 'Processed_Text']]), maxlen=max_sequence_length)
        movie_sequence = tf.convert_to_tensor(movie_sequence)  # 텐서로 변환
        predictions = predict(model, movie_sequence)[0].numpy()

        sim_scores = []
        for i in range(len(predictions)):
            sim_scores.append((i, predictions[i]))

        # 유사도 순으로 정렬
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # 상위 5개 추천
        sim_scores = sim_scores[:5]
        movie_indices = [i[0] for i in sim_scores]
        recommendations = data.iloc[movie_indices]

        # 시각화를 위한 유사도 점수 포함
        similarity_scores = [i[1] for i in sim_scores]
        return recommendations[['movie_name', 'genre', 'country_of_production', 'year_of_production']].values.tolist(), similarity_scores

    else:
        # 필터링된 영화에서 상위 5개 선택
        recommendations = data.head(5)
        similarity_scores = [1] * len(recommendations)  # 필터링된 결과에서는 유사도 점수를 1로 설정

    recommendations = recommendations.copy()
    recommendations.loc[:, 'year_of_production'] = recommendations['year_of_production'].astype(int)
    return recommendations[['movie_name', 'genre', 'country_of_production', 'year_of_production']].values.tolist(), similarity_scores

# 사용자 입력을 통한 영화 제목, 장르, 제작 국가, 제작 연도 입력
user_input = input("영화 제목, 장르, 제작 국가, 제작 연도를 쉼표로 구분하여 입력하세요 (예: 제목,장르,국가,연도): ").strip()

# 입력된 값을 쉼표로 구분하여 리스트로 변환
inputs = [i.strip() if i.strip() else None for i in user_input.split(',')]
inputs = inputs + [None] * (4 - len(inputs))  # 입력된 값의 개수가 4개가 되도록 보충
movie_title, genre, country, year = inputs

recommended_movies, similarity_scores = get_recommendations_dl(movie_title, genre, country, year)
print("추천된 영화들:\n", recommended_movies)


Epoch 1/100
95/95 [==============================] - 46s 259ms/step - loss: 0.4171 - accuracy: 0.0000e+00 - val_loss: 0.3942 - val_accuracy: 0.0000e+00
Epoch 2/100
95/95 [==============================] - 21s 222ms/step - loss: 0.3736 - accuracy: 0.0000e+00 - val_loss: 0.3530 - val_accuracy: 0.0000e+00
Epoch 3/100
95/95 [==============================] - 20s 215ms/step - loss: 0.3345 - accuracy: 0.0000e+00 - val_loss: 0.3160 - val_accuracy: 0.0000e+00
Epoch 4/100
95/95 [==============================] - 24s 251ms/step - loss: 0.2993 - accuracy: 3.3102e-04 - val_loss: 0.2827 - val_accuracy: 0.0000e+00
Epoch 5/100
95/95 [==============================] - 21s 219ms/step - loss: 0.2678 - accuracy: 3.3102e-04 - val_loss: 0.2528 - val_accuracy: 0.0000e+00
Epoch 6/100
95/95 [==============================] - 20s 208ms/step - loss: 0.2394 - accuracy: 0.0000e+00 - val_loss: 0.2259 - val_accuracy: 0.0000e+00
Epoch 7/100
95/95 [==============================] - 25s 260ms/step - loss: 0.2139 - acc